<a href="https://colab.research.google.com/github/stellagerantoni/learning-time-series-counterfactuals/blob/main/multivariate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 ! git clone https://github.com/stellagerantoni/learning-time-series-counterfactuals
 %cd learning-time-series-counterfactuals/

Cloning into 'learning-time-series-counterfactuals'...
remote: Enumerating objects: 309, done.
remote: Counting objects: 100% (171/171), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 309 (delta 120), reused 24 (delta 24), pack-reused 138
Receiving objects: 100% (309/309), 4.37 MiB | 8.85 MiB/s, done.
Resolving deltas: 100% (189/189), done.
/content/learning-time-series-counterfactuals


In [2]:

!pip install -q wildboar
!pip install -q scikit-learn
!pip install -q stumpy
!pip install -q fastdtw
!pip install aeon[all_extras]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.0/39.0 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.1/812.1 kB 71.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 102.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.4/153.4 kB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import logging
import os
import warnings
from argparse import ArgumentParser
from aeon.datasets import load_classification

from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from scipy.spatial import distance_matrix
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KDTree, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from wildboar.datasets import load_dataset
from wildboar.ensemble import ShapeletForestClassifier
from wildboar.explain.counterfactual import counterfactuals

from _composite import ModifiedLatentCF
%cd src
from _vanilla import LatentCF
from help_functions import (ResultWriter, conditional_pad, evaluate,
                            find_best_lr, plot_graphs,
                            reset_seeds, time_series_normalize,
                            time_series_revert, upsample_minority,
                            validity_score)
from keras_models import *

/content/learning-time-series-counterfactuals/src


In [4]:
os.environ['TF_DETERMINISTIC_OPS'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
RANDOM_STATE = 39

In [5]:

X, y, meta_data = load_classification("FaceDetection")
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)
print(X[:3])

 Shape of X =  (9414, 144, 62)
 Meta data =  {'problemname': 'facedetection', 'timestamps': False, 'missing': False, 'univariate': False, 'equallength': True, 'classlabel': True, 'targetlabel': False, 'class_values': ['0', '1']}
[[[-0.07545  -0.336703 -0.278238 ... -0.411078 -1.016122 -1.161735]
  [ 0.05608  -0.128013 -0.323847 ... -2.114348  0.208789 -0.509533]
  [-0.824537 -0.746068 -0.482871 ... -0.929275 -1.007972 -0.292018]
  ...
  [-0.56758  -1.073942 -1.136367 ... -0.541122 -0.765445 -1.73308 ]
  [-0.23404   0.104291  0.327425 ... -1.458801  0.318952  1.854007]
  [-0.356189 -0.511199 -0.483072 ... -1.177502 -0.728301 -0.400074]]

 [[-0.093192 -0.310616 -0.388905 ...  1.937879  0.978442  0.626839]
  [-1.584982 -1.138981 -0.852494 ... -0.334188  0.636669  0.078561]
  [-0.462859 -0.301394 -0.50619  ...  0.864706  0.113177 -0.337559]
  ...
  [-0.056147 -0.051994 -0.195028 ... -2.212319 -1.052363 -1.774344]
  [-0.71979  -0.624242 -0.548486 ...  0.276439  1.380659  0.376838]
  [-0.444

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

In [7]:
# Upsample the minority class
pos_label, neg_label = '1', '0'
y_train_copy = y_train.copy()

pos_counts = pd.value_counts(y_train)[pos_label]
neg_counts = pd.value_counts(y_train)[neg_label]
print(f"negative_count = {neg_counts}, positive_count = {pos_counts}")

if pos_counts!=neg_counts:
  X_train, y_train = upsample_minority(X_train, y_train, pos_label=pos_label, neg_label=neg_label)
  print(f"Data upsampling performed, current distribution of y: \n{pd.value_counts(y_train)}.")
else:
   print(f"Data upsampling not needed, current distribution of y: \n{pd.value_counts(y_train)}.")


negative_count = 3766, positive_count = 3765
Data upsampling performed, current distribution of y: 
1    3766
0    3766
dtype: int64.


In [8]:
data_reshaped = X_train.reshape(-1, 1)
data_reshaped

array([[ 0.226878],
       [ 0.496101],
       [ 0.470299],
       ...,
       [-0.58613 ],
       [-0.701873],
       [ 0.131871]])

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def time_series_normalize(data, n_timesteps, n_features, scaler=None):
    # First transpose the data to have shape (samples, timesteps, features)
    data_transposed = np.transpose(data, (0, 2, 1))

    # Then reshape data to have timesteps as rows for normalization
    data_reshaped = data_transposed.reshape(-1, n_features)

    if scaler is None:
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaler.fit(data_reshaped)

    normalized = scaler.transform(data_reshaped)

    # Return data reshaped
    normalized_transposed = normalized.reshape(-1, n_timesteps, n_features)
    return np.transpose(normalized_transposed, (0, 2, 1)), scaler

In [10]:
def conditional_pad_multivariate(X):
    num_timesteps = X.shape[2]

    if num_timesteps % 4 != 0:
        next_num = (int(num_timesteps / 4) + 1) * 4
        padding_size = next_num - num_timesteps
        X_padded = np.pad(
            X, pad_width=((0, 0), (0, 0), (0, padding_size))
        )

        return X_padded, padding_size

    return X, 0

In [11]:
n_training, n_features, n_timesteps = X_train.shape

X_train_processed, trained_scaler = time_series_normalize(data=X_train, n_timesteps=n_timesteps, n_features = n_features)
X_test_processed, _ = time_series_normalize(data=X_test, n_timesteps=n_timesteps, scaler=trained_scaler, n_features = n_features)

X_train_processed_padded, padding_size = conditional_pad(X_train_processed) # add extra padding zeros if n_timesteps cannot be divided by 4, required for 1dCNN autoencoder structure
X_test_processed_padded, _ = conditional_pad(X_test_processed)

n_timesteps_padded = X_train_processed_padded.shape[2]
print(f"Data pre-processed, original #timesteps={n_timesteps}, padded #timesteps={n_timesteps_padded}.")

Data pre-processed, original #timesteps=62, padded #timesteps=62.


In [12]:
y_train_classes = y_train
y_test_classes = y_test

y_train_classes

array(['1', '0', '1', ..., '0', '1', '1'], dtype='<U1')

In [13]:
y_train_classes = y_train
y_test_classes = y_test

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, len(np.unique(y_train)))
y_test = to_categorical(y_test, len(np.unique(y_test)))
print(f"y_train = \n\n{y_train}\n\n y_test = \n\n{y_test}")

y_train = 

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]

 y_test = 

[[0. 1.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


In [14]:
print(np.min(X_train_processed), np.max(X_train_processed))
print(np.min(X_train), np.max(X_train))

0.0 1.0000000000000002
-24.327769 24.326942


In [18]:
def Classifier(
    n_timesteps, n_features, n_conv_layers=1, add_dense_layer=True, n_output=1
):
    # https://keras.io/examples/timeseries/timeseries_classification_from_scratch/
    inputs = keras.Input(shape=(n_features, n_timesteps), dtype="float32")

    if add_dense_layer:
        x = keras.layers.Dense(128)(inputs)
    else:
        x = inputs

    for i in range(n_conv_layers):
        x = keras.layers.Conv1D(filters=64, kernel_size=3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.ReLU()(x)

    x = keras.layers.MaxPooling1D(pool_size=2, padding="same")(x)
    x = keras.layers.Flatten()(x)

    if n_output >= 2:
        outputs = keras.layers.Dense(n_output, activation="softmax")(x)
    else:
        outputs = keras.layers.Dense(1, activation="sigmoid")(x)

    classifier = keras.models.Model(inputs=inputs, outputs=outputs)

    return classifier

In [22]:
y_train_classes

array(['1', '0', '1', ..., '0', '1', '1'], dtype='<U1')

In [23]:
shallow_cnn = False
# ### 1dCNN classifier
if shallow_cnn == True:
    false(f"Check shallow_cnn argument={shallow_cnn}, use the shallow structure.")
    classifier = Classifier(n_timesteps_padded, n_features, n_conv_layers=1, add_dense_layer=True) # shallow CNN for small data size
else:
    classifier = Classifier(n_timesteps_padded, n_features, n_conv_layers=3, add_dense_layer=False) # deeper CNN layers for data with larger size

optimizer = keras.optimizers.Adam(lr=0.0001)
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define the early stopping criteria
early_stopping_accuracy = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=30, restore_best_weights=True)
# Train the model
reset_seeds()
classifier_history = classifier.fit(X_train_processed_padded,
        y_train_classes,
        epochs=150,
        batch_size=32,
        shuffle=True,
        verbose=True,
        validation_data=(X_test_processed_padded, y_test),
        callbacks=[early_stopping_accuracy])

Epoch 1/150


UnimplementedError: ignored